# MNIST Dataset ANN
**For the Intelligent systems course at the University of Applied Sciences.**

Authors
- Evert Kors, 500709685
- Mathijs Haakman, 500718568


### Het laden van de MNIST data

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

### Het spliten van de training set
We splitsen de data op in verschillende sets, training, validation and testing. 
- Training word gebruikt om de gewichten aan te passen in het neural net.
- Validation word gebruikt om het neural net te fine tunen.
- Test word gebruikt om de kwaliteit te testen

In [1]:
n_train = mnist.train.num_examples # 55,000
n_validation = mnist.validation.num_examples # 5000
n_test = mnist.test.num_examples # 10,000

NameError: name 'mnist' is not defined

### Bepalen van de ANN architectuur
Het artifical neural network bestaat uit 3 hidden layers, 1 input layer, 1 output layer.

In [2]:
n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)

### Bepalen van de learning variables
- learning_rate word gebruikt om de stappen van het model te bepalem. als het te groot is dan zal het model nooit inzoomen op de beste waarde. Als het te klein is, dan zal het nooit de beste waarde bereiken.
- n_iterations word gebruikt om te bepalen hoeveel iteraties het ANN moet doen.
- batch_size word gebruikt voor het aantal examples per iteratie. (per learning cycle)
- dropout is het percentage wat eruit word gegooid bij de laatste laag, dit is voor het voorkomen van overfitting.

In [ ]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

### Bepalen van TensorFlow tensors

In [ ]:
# Input for every pixel in an image
X = tf.placeholder("float", [None, n_input])
# Output for the labels 0,1,2,3,4,5,6,7,8,9
Y = tf.placeholder("float", [None, n_output])
# To control the dropout rate
keep_prob = tf.placeholder(tf.float32)

### Startgewichten
De startgewichten worden geiniteerd met een random waarde, dichtbij 0. Zodat ze in de positieve en negatieve richting kunnen veranderen.

In [ ]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

### Toevoegen van biases
We gebruiken kleine waardes voor de biases

In [ ]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

### De layers toevoegen

In [ ]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

### Meer stappen

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

### De metrieken om de training te evalueren

In [ ]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Init sesssion

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

### Voeden van het model

In [1]:
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

NameError: name 'n_iterations' is not defined

In [ ]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
print("\nAccuracy on test set:", test_accuracy)